In [1]:
import numpy as np
import scanorama
import scipy.sparse as sparse
import pandas as pd

/home/dblux/anaconda3/envs/r-1/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
# # Equal-sized batches
# num_batches = 6

# # Odd-sized batches (MAQC)
# odd_batches = [10,9,9,8,8,7]
# odd_cutpoints = np.cumsum(odd_batches)[:-1]

# log_maqc = np.loadtxt("data/scanorama/log_maqc.tsv")
# print(log_maqc[:5,:5])
# print(log_maqc.shape)

[ 39 114 156 187 230 263 290 345 405]


In [3]:
# Transpose array into samples x genes
list_arr = np.split(np.transpose(log_maqc), num_batches, axis = 0)
# # When batches are in odd proportions
# list_arr = np.split(np.transpose(log_maqc), odd_cutpoints, axis = 0)
for elem in list_arr:
    print(elem.shape)

(10, 54675)
(10, 54675)
(10, 54675)
(10, 54675)
(10, 54675)
(10, 54675)


In [4]:
file = "data/scanorama/probeset_names.txt"
with open(file, "r") as f:
    list_genes = f.readlines()

list_genes = [gene.strip() for gene in list_genes]
# Repeat list of genes for each batch
list_list_genes = [list_genes] * num_batches
print(len(list_list_genes))

6


In [5]:
# # Integration (Returns SVD embeddings)
# integrated, genes = scanorama.integrate(list_arr, list_list_genes)

# Batch correction.
list_corrected_arr, gene_colnames = scanorama.correct(list_arr, list_list_genes, knn = 10)

# # Integration and batch correction.
# integrated, corrected, genes = scanorama.correct(list_arr, list_list_genes, return_dimred=True)

Found 54675 genes among all datasets
[[0.  1.  0.3 1.  0.5 0.4]
 [0.  0.  1.  0.8 0.9 0.9]
 [0.  0.  0.  0.9 1.  1. ]
 [0.  0.  0.  0.  0.8 1. ]
 [0.  0.  0.  0.  0.  0.6]
 [0.  0.  0.  0.  0.  0. ]]
Processing datasets (3, 5)
Processing datasets (2, 5)
Processing datasets (2, 4)
Processing datasets (1, 2)
Processing datasets (0, 3)
Processing datasets (0, 1)
Processing datasets (2, 3)
Processing datasets (1, 5)
Processing datasets (1, 4)
Processing datasets (3, 4)
Processing datasets (1, 3)
Processing datasets (4, 5)
Processing datasets (0, 4)
Processing datasets (0, 5)
Processing datasets (0, 2)


In [6]:
corrected_arr = sparse.vstack(list_corrected_arr)
corrected_arr1 = sparse.csr_matrix.transpose(corrected_arr)
print(corrected_arr1.shape)
# Replace negative values with 0
corrected_arr1[corrected_arr1 < 0] = 0
corrected_df = pd.DataFrame(corrected_arr1.toarray())
# Name rownames of df with output list from scanorama
corrected_df.index = gene_colnames
# Rows of genes not ordered the same way as input
# Sort rows according to initial rownames
corrected_df1 = corrected_df.reindex(list_genes)
print(corrected_df1.iloc[:5,:5])

(54675, 60)
                  0         1         2         3         4
1007_s_at  0.006693  0.006478  0.006609  0.006641  0.006574
1053_at    0.006494  0.006402  0.006388  0.006498  0.006433
117_at     0.000157  0.000227  0.000000  0.000000  0.000001
121_at     0.005231  0.005113  0.005167  0.005188  0.005151
1255_g_at  0.000000  0.000000  0.000000  0.000000  0.000000


In [7]:
corrected_df1.to_csv("data/scanorama/scanorama_data_k10.tsv",
                     sep = "\t", header = False)

In [8]:
help(scanorama.correct)

Help on function correct in module scanorama.scanorama:

correct(datasets_full, genes_list, return_dimred=False, batch_size=5000, verbose=2, ds_names=None, dimred=100, approx=True, sigma=15, alpha=0.1, knn=20, return_dense=False, hvg=None, union=False, geosketch=False, geosketch_max=20000, seed=0)
    Integrate and batch correct a list of data sets.
    
    Parameters
    ----------
    datasets_full : `list` of `scipy.sparse.csr_matrix` or of `numpy.ndarray`
        Data sets to integrate and correct.
    genes_list: `list` of `list` of `string`
        List of genes for each data set.
    return_dimred: `bool`, optional (default: `False`)
        In addition to returning batch corrected matrices, also returns
        integrated low-dimesional embeddings.
    batch_size: `int`, optional (default: `5000`)
        The batch size used in the alignment vector computation. Useful when
        correcting very large (>100k samples) data sets. Set to large value
        that runs within avai